# Praca domowa 4
#### Autor: Bartosz Sawicki

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from dalex import datasets

from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import  RandomizedSearchCV, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.svm import SVR, SVC


## Apartments

In [ ]:
apartments = pd.concat([datasets.load_apartments(),datasets.load_apartments_test()])

In [ ]:
ap_X_train, ap_X_test, ap_y_train, ap_y_test = train_test_split(
    apartments.drop(['m2_price'], axis=1), 
    apartments.loc[:,'m2_price'], 
    random_state=123)

### EDA

In [ ]:
apartments.info()

In [ ]:
sns.histplot(ap_y_train)

Zmienna celu ma lekko prawoskośny rozkład, ale nie powinno być to problemem w zadaniu regresji.

In [ ]:
sns.pairplot(ap_X_train)

Wszystkie zmienne mają dosyć równomierny rozkład, liczba pokoji jest skorelowana z powierzchnią

In [ ]:
ap_train = pd.concat([ap_X_train, ap_y_train], axis=1)
sns.heatmap(ap_train.corr(), annot=True)

Usuniemy jedną ze skorelowanych zmiennych

In [ ]:
ap_X_test.drop(['no_rooms'], axis=1, inplace=True)
ap_X_train.drop(['no_rooms'], axis=1, inplace=True)

## Mobile price classification 
Źródło: [https://www.kaggle.com/iabhishekofficial/mobile-price-classification](https://www.kaggle.com/iabhishekofficial/mobile-price-classification). 

Zbiór danych o modelach telefonów komórkowych z etykietą kategorii cenowej. Należy przewidzieć do jakiej kategorii należy telefon. 

In [ ]:
mobiles_train = pd.read_csv('data/train.csv')
mobiles_test = pd.read_csv('data/test.csv')

In [ ]:
mobiles_test.columns

In [ ]:
mobiles_train.columns

W zbiorze danych `test` brakuje zmiennej celu, więc jest dla nas bezużyteczny. Podzielimy zbiór train na treningowy i testowy.

In [ ]:
mobiles = pd.read_csv('data/train.csv')
mob_X = mobiles.drop(['price_range'], axis=1)
mob_y = mobiles.loc[:,'price_range']

In [ ]:
mob_X_train, mob_X_test, mob_y_train, mob_y_test = train_test_split(mob_X, mob_y, random_state=123, stratify=mob_y)

### EDA

In [ ]:
mob_X_train.info()

In [ ]:
sns.histplot(mob_y_train)

In [ ]:
mob_train = pd.concat([mob_X_train, mob_y_train], axis=1)
fig, ax = plt.subplots(figsize=(15,12))

sns.heatmap(ax=ax, data=mob_train.corr())

- pc skorelowany z fc (pc - primary camera, fc - front camera). Rozdzielczość aparatów podstawowego i przedniego.
- px_height i px_width skorelowane (szer. i wys. ekranu w pixelach)
- Na cenę ma wpływ: 
    + pamięć RAM
    + pojemność baterii
    + rozdzielczość ekranu

In [ ]:
cols = ['battery_power', 'fc', 'int_memory', 'pc', 'px_height', 'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time']
sns.pairplot(mob_X_train.loc[:,cols])

Usuniemy z modelu skorelowane zmienne

In [ ]:
mob_X_test = mob_X_test.drop(['fc', 'px_height', 'sc_h'], axis=1)
mob_X_train = mob_X_train.drop(['fc', 'px_height', 'sc_h'], axis=1)

## Tworzenie modeli
### Apartments

In [ ]:
ap_scaling_clf = make_pipeline(OneHotEncoder(sparse=False), StandardScaler(), SVR())
ap_clf = make_pipeline(OneHotEncoder(), SVR())

ap_scaling_clf.fit(ap_X_train, ap_y_train)
ap_clf.fit(ap_X_train, ap_y_train)

In [ ]:
display(mean_squared_error(ap_y_test, ap_scaling_clf.predict(ap_X_test)))
display(mean_squared_error(ap_y_test, ap_clf.predict(ap_X_test)))

SVM Regressor osiągnął mniejszy MSE, gdy nie skalował danych. Zaprzecza to tezie postawionej w artykule.

### Mobiles

In [ ]:
mob_scaling_clf = make_pipeline(StandardScaler(), SVC())
mob_clf = SVC()

mob_scaling_clf.fit(mob_X_train, mob_y_train)
mob_clf.fit(mob_X_train, mob_y_train)

In [ ]:
display(accuracy_score(mob_y_test, mob_scaling_clf.predict(mob_X_test)))
display(accuracy_score(mob_y_test, mob_clf.predict(mob_X_test)))

Podobnie jest w tym przypadku. Lepszy wynik uzyskuje klasyfikator bez skalowania.

## Tuning hiperparametrów
### Apartments

In [ ]:
ap_scaling_clf.get_params().keys()

In [ ]:
distributions = dict(svr__gamma=['scale', 'auto', .05, .1, .25],
                    svr__C=[.01, .05, .1, .25, .5, 2, 5],
                    svr__kernel=['poly'],
                    svr__degree=[x for x in range(1,6)])
ap_scaling_clf_rs = RandomizedSearchCV(ap_scaling_clf, distributions, 
                                       random_state=123, scoring='neg_mean_squared_error', 
                                       n_jobs=-1, verbose=5, n_iter=5)
search = ap_scaling_clf_rs.fit(ap_X_train, ap_y_train)
search.best_params_

In [ ]:
ap_best_clf = search.best_estimator_
display(mean_squared_error(ap_y_test, ap_best_clf.predict(ap_X_test)))

Uzyskaliśmy dużą poprawę MSE dla SVR (z 836907 do 3824)

### Mobiles

In [ ]:
distributions = dict(svc__gamma=['scale', 'auto', .05, .1, .25],
                    svc__C=[.01, .05, .1, .25, .5, 2, 5, 10, 25],
                    svc__kernel=['poly'],
                    svc__degree=[x for x in range(1,6)])
mob_scaling_clf_rs = RandomizedSearchCV(mob_scaling_clf, distributions, 
                                       random_state=123, scoring='accuracy', 
                                       n_jobs=-1, verbose=5)
mob_search = mob_scaling_clf_rs.fit(mob_X_train, mob_y_train)
mob_search.best_params_

In [ ]:
mob_best_clf = mob_search.best_estimator_
display(accuracy_score(mob_y_test, mob_best_clf.predict(mob_X_test)))

Poprawiliśmy skuteczność z 83% do prawie 89%

## Wnioski

Zastosowanie tuningu hiperparametrów może znacząco poprawić wynik modelu. RandomizedSearchCV pozwala na szybkie sprawdzenie różnych kombinacji hiperparametrów, które póżniej można udoskonalić poprzez GridSearch